In [4]:
import json
from pathlib import Path

import numpy as np
import SimpleITK
from PIL import Image
from tqdm.auto import tqdm

DATASETS_DIR = Path("/rodata/mnradonc_dev/m299164/trackrad/datasets")

nnUNet_raw = DATASETS_DIR / "nnUNet/nnUNet_raw"
nnUNet_preprocessed = DATASETS_DIR / "nnUNet/nnUNet_preprocessed"
nnUNet_results = DATASETS_DIR / "nnUNet/nnUNet_results"

INPUT_DIR = DATASETS_DIR / "trackrad2025/trackrad2025_labeled_training_data"
SUGGESTED_MASKS_DIR = Path("./train-unet")

In [6]:
dataset_idx = 473
dataset_name = "TrackRadSmooth15"
frame_count = 15
assert frame_count >= 1

case_dirs = sorted(case for case in INPUT_DIR.iterdir())
count = 0
DAVIS_PALETTE = b"\x00\x00\x00\x80\x00\x00\x00\x80\x00\x80\x80\x00\x00\x00\x80\x80\x00\x80\x00\x80\x80\x80\x80\x80@\x00\x00\xc0\x00\x00@\x80\x00\xc0\x80\x00@\x00\x80\xc0\x00\x80@\x80\x80\xc0\x80\x80\x00@\x00\x80@\x00\x00\xc0\x00\x80\xc0\x00\x00@\x80\x80@\x80\x00\xc0\x80\x80\xc0\x80@@\x00\xc0@\x00@\xc0\x00\xc0\xc0\x00@@\x80\xc0@\x80@\xc0\x80\xc0\xc0\x80\x00\x00@\x80\x00@\x00\x80@\x80\x80@\x00\x00\xc0\x80\x00\xc0\x00\x80\xc0\x80\x80\xc0@\x00@\xc0\x00@@\x80@\xc0\x80@@\x00\xc0\xc0\x00\xc0@\x80\xc0\xc0\x80\xc0\x00@@\x80@@\x00\xc0@\x80\xc0@\x00@\xc0\x80@\xc0\x00\xc0\xc0\x80\xc0\xc0@@@\xc0@@@\xc0@\xc0\xc0@@@\xc0\xc0@\xc0@\xc0\xc0\xc0\xc0\xc0 \x00\x00\xa0\x00\x00 \x80\x00\xa0\x80\x00 \x00\x80\xa0\x00\x80 \x80\x80\xa0\x80\x80`\x00\x00\xe0\x00\x00`\x80\x00\xe0\x80\x00`\x00\x80\xe0\x00\x80`\x80\x80\xe0\x80\x80 @\x00\xa0@\x00 \xc0\x00\xa0\xc0\x00 @\x80\xa0@\x80 \xc0\x80\xa0\xc0\x80`@\x00\xe0@\x00`\xc0\x00\xe0\xc0\x00`@\x80\xe0@\x80`\xc0\x80\xe0\xc0\x80 \x00@\xa0\x00@ \x80@\xa0\x80@ \x00\xc0\xa0\x00\xc0 \x80\xc0\xa0\x80\xc0`\x00@\xe0\x00@`\x80@\xe0\x80@`\x00\xc0\xe0\x00\xc0`\x80\xc0\xe0\x80\xc0 @@\xa0@@ \xc0@\xa0\xc0@ @\xc0\xa0@\xc0 \xc0\xc0\xa0\xc0\xc0`@@\xe0@@`\xc0@\xe0\xc0@`@\xc0\xe0@\xc0`\xc0\xc0\xe0\xc0\xc0\x00 \x00\x80 \x00\x00\xa0\x00\x80\xa0\x00\x00 \x80\x80 \x80\x00\xa0\x80\x80\xa0\x80@ \x00\xc0 \x00@\xa0\x00\xc0\xa0\x00@ \x80\xc0 \x80@\xa0\x80\xc0\xa0\x80\x00`\x00\x80`\x00\x00\xe0\x00\x80\xe0\x00\x00`\x80\x80`\x80\x00\xe0\x80\x80\xe0\x80@`\x00\xc0`\x00@\xe0\x00\xc0\xe0\x00@`\x80\xc0`\x80@\xe0\x80\xc0\xe0\x80\x00 @\x80 @\x00\xa0@\x80\xa0@\x00 \xc0\x80 \xc0\x00\xa0\xc0\x80\xa0\xc0@ @\xc0 @@\xa0@\xc0\xa0@@ \xc0\xc0 \xc0@\xa0\xc0\xc0\xa0\xc0\x00`@\x80`@\x00\xe0@\x80\xe0@\x00`\xc0\x80`\xc0\x00\xe0\xc0\x80\xe0\xc0@`@\xc0`@@\xe0@\xc0\xe0@@`\xc0\xc0`\xc0@\xe0\xc0\xc0\xe0\xc0  \x00\xa0 \x00 \xa0\x00\xa0\xa0\x00  \x80\xa0 \x80 \xa0\x80\xa0\xa0\x80` \x00\xe0 \x00`\xa0\x00\xe0\xa0\x00` \x80\xe0 \x80`\xa0\x80\xe0\xa0\x80 `\x00\xa0`\x00 \xe0\x00\xa0\xe0\x00 `\x80\xa0`\x80 \xe0\x80\xa0\xe0\x80``\x00\xe0`\x00`\xe0\x00\xe0\xe0\x00``\x80\xe0`\x80`\xe0\x80\xe0\xe0\x80  @\xa0 @ \xa0@\xa0\xa0@  \xc0\xa0 \xc0 \xa0\xc0\xa0\xa0\xc0` @\xe0 @`\xa0@\xe0\xa0@` \xc0\xe0 \xc0`\xa0\xc0\xe0\xa0\xc0 `@\xa0`@ \xe0@\xa0\xe0@ `\xc0\xa0`\xc0 \xe0\xc0\xa0\xe0\xc0``@\xe0`@`\xe0@\xe0\xe0@``\xc0\xe0`\xc0`\xe0\xc0\xe0\xe0\xc0"
dataset_dir = nnUNet_raw / f"Dataset{dataset_idx}_{dataset_name}"
dataset_dir.mkdir(parents=True, exist_ok=True)
for train_case in tqdm(case_dirs):
    dataset_json = dataset_dir / "dataset.json"
    images_dir = dataset_dir / "imagesTr"
    labels_dir = dataset_dir / "labelsTr"

    images_dir.mkdir(parents=True, exist_ok=True)
    labels_dir.mkdir(parents=True, exist_ok=True)

    images_file = train_case / "images" / f"{train_case.name}_frames.mha"
    labels_file = train_case / "targets" / f"{train_case.name}_labels.mha"

    images_array = SimpleITK.GetArrayFromImage(SimpleITK.ReadImage(str(images_file)))

    labels_array = SimpleITK.GetArrayFromImage(SimpleITK.ReadImage(str(labels_file)))
    suggested_mask_pngs = (SUGGESTED_MASKS_DIR / train_case.name / "annotations").glob(
        "*.png"
    )
    suggested_mask_pngs = sorted(suggested_mask_pngs)

    assert images_array.shape[-1] == labels_array.shape[-1] == len(suggested_mask_pngs)

    for i in range(images_array.shape[-1]):
        case_id = f"{train_case.name}-{i:04d}"
        for j in range(frame_count):
            frame_idx = i - (frame_count - 1) + j
            # Repeat frames if we're at the beginning
            if frame_idx < 0:
                frame_idx = 0
            image = images_array[..., frame_idx].squeeze()
            suggested_mask = Image.open(suggested_mask_pngs[frame_idx])
            suggested_mask = np.array(suggested_mask)

            raw_image_path = images_dir / f"{case_id}_{2 * j:04d}.png"
            suggested_mask_path = images_dir / f"{case_id}_{2 * j + 1:04d}.png"

            img = Image.fromarray(image).save(raw_image_path)
            mask = Image.fromarray(suggested_mask)
            # mask.putpalette(DAVIS_PALETTE)
            mask.save(suggested_mask_path)

        label_png_path = labels_dir / f"{case_id}.png"
        label = Image.fromarray(labels_array[..., i])
        # label.putpalette(DAVIS_PALETTE)
        label.save(label_png_path)

        count += 1

    channel_names = ["raw_image", "suggested_mask"] * frame_count
    metadata = {
        "channel_names": {str(i): name for i, name in enumerate(channel_names)},
        "labels": {
            "background": 0,
            "lesion": 1,
        },
        "numTraining": count,
        "file_ending": ".png",
    }
    dataset_json.write_text(json.dumps(metadata, indent=4))

  0%|          | 0/50 [00:00<?, ?it/s]